# Data Cleaning
## Loading Data

In [326]:
import pandas as pd

In [327]:
disbursements_df = pd.read_pickle("exports/disbursements.pkl")
repayments_df = pd.read_pickle("exports/repayments.pkl")

In [328]:
disbursements_df.head()

,customer,date,tenure,account,amount,fee
0,91810ca1aa097db79f050f38e9946fa5482b4e28c925e2...,2024-03-19,14 days,3O66YENWELA6E2H1R9YLX0LDZNOMNHD4,360,43.2
1,42ca06e6fe1ff9803e82a5c20184671b54090e488f78d6...,2024-03-19,7 days,6XWHXKKR1W2HIA8I0V75PZFZBXUUGSVO,70,7.0
2,b23747f53af805e18ad16a4ef235b6642d88f9134644ff...,2024-03-19,7 days,OCGK3RJZ91A999VXD4VB3LATPSME3J5L,3500,350.0
3,1bd32f9b083fc6ddfffd65730fbfa66654fa76a19b0b0e...,2024-03-19,14 days,9X3Q682DOR7927IMMJLFHBGP0RP7YF5C,3500,420.0
4,e7cfbaa97ba7702c52df5f1dddba54bd26923ebad945f1...,2024-03-19,7 days,AQH88NNF8S76MGJL4J4ULEAE18O0KLWH,120,12.0


### Engineering due column combing loan amount plus fee

In [329]:
disbursements_df["due"] = disbursements_df.amount + disbursements_df.fee
disbursements_df.due = disbursements_df.due.round(2)
disbursements_df

,customer,date,tenure,account,amount,fee,due
0,91810ca1aa097db79f050f38e9946fa5482b4e28c925e2...,2024-03-19,14 days,3O66YENWELA6E2H1R9YLX0LDZNOMNHD4,360,43.2,403.2
1,42ca06e6fe1ff9803e82a5c20184671b54090e488f78d6...,2024-03-19,7 days,6XWHXKKR1W2HIA8I0V75PZFZBXUUGSVO,70,7.0,77.0
2,b23747f53af805e18ad16a4ef235b6642d88f9134644ff...,2024-03-19,7 days,OCGK3RJZ91A999VXD4VB3LATPSME3J5L,3500,350.0,3850.0
3,1bd32f9b083fc6ddfffd65730fbfa66654fa76a19b0b0e...,2024-03-19,14 days,9X3Q682DOR7927IMMJLFHBGP0RP7YF5C,3500,420.0,3920.0
4,e7cfbaa97ba7702c52df5f1dddba54bd26923ebad945f1...,2024-03-19,7 days,AQH88NNF8S76MGJL4J4ULEAE18O0KLWH,120,12.0,132.0
...,...,...,...,...,...,...,...
26570,b23747f53af805e18ad16a4ef235b6642d88f9134644ff...,2024-04-03,7 days,8ETPAYW0XG3UIO2RF8IJTBKR87SAM7KE,3500,350.0,3850.0
26571,9383d85b9a40ec42956bd9f5cb3c5b986a7d1113f9513d...,2024-04-03,14 days,FWMDBC7C7XDDI1ZODUUTRLRPSVB0WQWP,670,80.4,750.4
26572,934baee90b08c1cdbbb487577699b96968fdc8ad29065c...,2024-04-03,14 days,3S4JYCX5V3EAOPHFJG6UF4J2O3TZS1R9,150,18.0,168.0
26573,ddb2265ddb86bd8242002aecd0c248dce855551beb01d6...,2024-04-03,14 days,LDP1P2GVJ4Y7WTCLQ97V5SXDIPZAHM3U,150,18.0,168.0


In [330]:
repayments_df.head()

,date,customer,amount,month,type
0,27-JUN-24 07.16.36.000000000 AM,683131338d401fda38410a808797b7706bc3e364d0fe77...,500.65,202406,Automatic
1,27-JUN-24 05.26.50.000000000 PM,56a41251185bd205961556399289804607ccc660392837...,2833.33,202406,Automatic
2,27-JUN-24 06.45.40.000000000 PM,000e57e83f161e4ba6458b3e32c00815405c5a005e652b...,143.98,202406,Automatic
3,27-JUN-24 01.24.57.000000000 PM,f3606d5ac9a89e0251a60d7183a09bc742eba2e36882af...,1000.00,202406,Automatic
4,27-JUN-24 01.31.44.000000000 PM,f3606d5ac9a89e0251a60d7183a09bc742eba2e36882af...,801.00,202406,Manual


## Standardizing Formats

In [331]:
def parse_mixed_dates(date):
    try:
        return pd.to_datetime(date, format="%d-%b-%y %I.%M.%S.%f %p")  # for "14-JUN-24 04.12.23.000000000 PM"
    except ValueError:
        try:
            return pd.to_datetime(date, format="%m/%d/%Y")  # for "07/18/2024"
        except ValueError:
            return pd.NaT  # If neither works, mark as NaT

In [332]:
repayments_df.date = repayments_df.date.apply(parse_mixed_dates)
repayments_df.date = repayments_df.date.dt.date
repayments_df.date = pd.to_datetime(repayments_df.date)
repayments_df.head()

,date,customer,amount,month,type
0,2024-06-27,683131338d401fda38410a808797b7706bc3e364d0fe77...,500.65,202406,Automatic
1,2024-06-27,56a41251185bd205961556399289804607ccc660392837...,2833.33,202406,Automatic
2,2024-06-27,000e57e83f161e4ba6458b3e32c00815405c5a005e652b...,143.98,202406,Automatic
3,2024-06-27,f3606d5ac9a89e0251a60d7183a09bc742eba2e36882af...,1000.00,202406,Automatic
4,2024-06-27,f3606d5ac9a89e0251a60d7183a09bc742eba2e36882af...,801.00,202406,Manual


In [335]:
repayments_df.isna().sum()

date        0
customer    0
amount      0
month       0
type        0
dtype: int64

In [336]:
repayments_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 65905 entries, 0 to 66004
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      65905 non-null  datetime64[ns]
 1   customer  65905 non-null  object        
 2   amount    65905 non-null  float64       
 3   month     65905 non-null  int64         
 4   type      65905 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 3.0+ MB


In [337]:
repayments_df.type = repayments_df.type.str.lower().str.strip()
repayments_df.type.unique()

array(['automatic', 'manual'], dtype=object)

### Dropping Redundant Month Column

In [338]:
repayments_df.drop("month", axis=1, inplace=True)
repayments_df.head()

,date,customer,amount,type
0,2024-06-27,683131338d401fda38410a808797b7706bc3e364d0fe77...,500.65,automatic
1,2024-06-27,56a41251185bd205961556399289804607ccc660392837...,2833.33,automatic
2,2024-06-27,000e57e83f161e4ba6458b3e32c00815405c5a005e652b...,143.98,automatic
3,2024-06-27,f3606d5ac9a89e0251a60d7183a09bc742eba2e36882af...,1000.00,automatic
4,2024-06-27,f3606d5ac9a89e0251a60d7183a09bc742eba2e36882af...,801.00,manual


### Formatting tenure length

In [339]:
disbursements_df["tenure"] = disbursements_df["tenure"].str.extract("(\d+)").astype(int)
disbursements_df.head()

,customer,date,tenure,account,amount,fee,due
0,91810ca1aa097db79f050f38e9946fa5482b4e28c925e2...,2024-03-19,14,3O66YENWELA6E2H1R9YLX0LDZNOMNHD4,360,43.2,403.2
1,42ca06e6fe1ff9803e82a5c20184671b54090e488f78d6...,2024-03-19,7,6XWHXKKR1W2HIA8I0V75PZFZBXUUGSVO,70,7.0,77.0
2,b23747f53af805e18ad16a4ef235b6642d88f9134644ff...,2024-03-19,7,OCGK3RJZ91A999VXD4VB3LATPSME3J5L,3500,350.0,3850.0
3,1bd32f9b083fc6ddfffd65730fbfa66654fa76a19b0b0e...,2024-03-19,14,9X3Q682DOR7927IMMJLFHBGP0RP7YF5C,3500,420.0,3920.0
4,e7cfbaa97ba7702c52df5f1dddba54bd26923ebad945f1...,2024-03-19,7,AQH88NNF8S76MGJL4J4ULEAE18O0KLWH,120,12.0,132.0


## Merging Data

In [340]:
disbursements_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26542 entries, 0 to 26574
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   customer  26542 non-null  object        
 1   date      26542 non-null  datetime64[ns]
 2   tenure    26542 non-null  int64         
 3   account   26542 non-null  object        
 4   amount    26542 non-null  int64         
 5   fee       26542 non-null  float64       
 6   due       26542 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 1.6+ MB


In [341]:
repayments_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 65905 entries, 0 to 66004
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      65905 non-null  datetime64[ns]
 1   customer  65905 non-null  object        
 2   amount    65905 non-null  float64       
 3   type      65905 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 2.5+ MB


In [342]:
merged_df = disbursements_df.merge(repayments_df, on=["customer"], how="left")
merged_df.rename(columns={"date_x": "disbursement_date", "date_y": "repayment_date"}, inplace=True)
merged_df.rename(columns={"amount_x": "disbursement_amount", "amount_y": "repayment_amount"}, inplace=True)
merged_df

,customer,disbursement_date,tenure,account,disbursement_amount,fee,due,repayment_date,repayment_amount,type
0,91810ca1aa097db79f050f38e9946fa5482b4e28c925e2...,2024-03-19,14,3O66YENWELA6E2H1R9YLX0LDZNOMNHD4,360,43.2,403.2,2024-06-11,38.89,automatic
1,91810ca1aa097db79f050f38e9946fa5482b4e28c925e2...,2024-03-19,14,3O66YENWELA6E2H1R9YLX0LDZNOMNHD4,360,43.2,403.2,2024-06-11,77.77,manual
2,91810ca1aa097db79f050f38e9946fa5482b4e28c925e2...,2024-03-19,14,3O66YENWELA6E2H1R9YLX0LDZNOMNHD4,360,43.2,403.2,2024-06-25,425.60,manual
3,91810ca1aa097db79f050f38e9946fa5482b4e28c925e2...,2024-03-19,14,3O66YENWELA6E2H1R9YLX0LDZNOMNHD4,360,43.2,403.2,2024-06-09,0.55,automatic
4,91810ca1aa097db79f050f38e9946fa5482b4e28c925e2...,2024-03-19,14,3O66YENWELA6E2H1R9YLX0LDZNOMNHD4,360,43.2,403.2,2024-06-09,16.67,automatic
...,...,...,...,...,...,...,...,...,...,...
766241,c2fc4aec21a377e3e03c7f47575097a44fd9724a6dfadc...,2024-04-03,14,O17J6XD4E0GUR7R7GPPAEZ43CL1AC07D,650,78.0,728.0,2024-04-17,15.00,automatic
766242,c2fc4aec21a377e3e03c7f47575097a44fd9724a6dfadc...,2024-04-03,14,O17J6XD4E0GUR7R7GPPAEZ43CL1AC07D,650,78.0,728.0,2024-04-17,1.94,automatic
766243,c2fc4aec21a377e3e03c7f47575097a44fd9724a6dfadc...,2024-04-03,14,O17J6XD4E0GUR7R7GPPAEZ43CL1AC07D,650,78.0,728.0,2024-04-24,1.00,automatic
766244,c2fc4aec21a377e3e03c7f47575097a44fd9724a6dfadc...,2024-04-03,14,O17J6XD4E0GUR7R7GPPAEZ43CL1AC07D,650,78.0,728.0,2024-05-04,0.06,automatic


## Write To File

In [343]:
disbursements_df.to_pickle("exports/disbursements_clean.pkl")

In [344]:
repayments_df.to_pickle("exports/repayments_clean.pkl")

In [345]:
merged_df.to_pickle("exports/merged.pkl")